# Homework 3 Solutions

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

## Question 1

The `titanic.xls` spreadsheet in the `data` directory contains data regarding the passengers on the Titanic when it sank in 1912. A recent [Kaggle competition](http://www.kaggle.com/c/titanic-gettingStarted) was based on predicting survival for passengers based on the attributes in the passenger list. 

Use scikit-learn to build both a support vector classifier and a logistic regression model to predict survival on the Titanic. Use cross-validation to assess your models, and try to tune them to improve performance.

Discuss the benefits and drawbacks of both approaches for application to such problems.

In [ ]:
titanic = pd.read_excel("/Users/schluetd/Bios8366/data/titanic.xls", "titanic")
# Redefine gender as a numeric variable
titanic['male'] = titanic.sex.replace({'male':1, 'female':0})
titanic_subset = titanic.filter(items = ('survived','pclass','male','age','sibsp','parch','fare')).copy()

# Percentages of missing by variable
titanic_subset.isnull().sum()/len(titanic_subset)*100

Based on the above, the variables `age` and `fare` have missing values present. (Note that there is only one observation missing `fare`, hence the low percent missingness. We could probably just drop that observation, but since we will be imputing age anyway, we'll also impute fare for that observation.)  

We will perform multiple imputation (MI). MI works best when the data are missing at random, or when the missingness mechanism is a function of other data to which we have access. Looking at the correlation matrix below, we can get a sense of whether or not we might be reasonably able to predict age and fare from the other variables in the titanic dataset. We use Spearman rank correlation since class and sex are categorical. Some correlations are particularly large in absolute value, such as between fare and pclass. (We would expect this since a higher fare would be associated with first class compared to third class.)

In [ ]:
# Use spearman correlation since there are categorical variables
titanic_subset.dropna().corr(method='spearman')


To perform MI, we will use regularized regression to predict age and fare from the remaining variables. Our outcome is included in the model, which is considered best practice to preserve any association between the imputed predictors and the outcome (as noted in *Regression Modeling Strategies*). For this, we will use ElasticNet and vary the ratio of L1 (Lasso) vs L2 (Ridge) regularization. Missing values for a given observation will be replaced with the average of the imputed values for that observation. This takes into account some of the uncertainty involved with imputation.

In [ ]:
from sklearn.linear_model import ElasticNet

# Only use complete variables to perform imputation - pop off the two variables to be imputed
impute_subset = titanic_subset.copy()
# Variables we are imputing
imp_age = impute_subset.pop('age')
imp_fare = impute_subset.pop('fare')

# Set up lists to hold imputations
age_imp = []
fare_imp = []

# Determine indexes associated with missing values
missing_age = np.isnan(imp_age)
missing_fare = np.isnan(imp_fare)

# Vary the ratio of L1 (Lasso) and L2 (Ridge) regularization used
for l1_l2_ratio in 0.1, 0.25, 0.5, 0.75, 0.9:
    
    # Fit imputation model for age on the non-missing data
    mod_age = ElasticNet(l1_ratio=l1_l2_ratio)
    mod_age.fit(impute_subset[~missing_age], imp_age[~missing_age])
    # Predict the missing values
    imputed_age = mod_age.predict(impute_subset[missing_age])
    age_imp.append(imputed_age)

    # Fit imputation model for age on the non-missing data
    mod_fare = ElasticNet(l1_ratio=l1_l2_ratio)
    mod_fare.fit(impute_subset[~missing_fare], imp_fare[~missing_fare])
    # Predict the missing values
    imputed_fare = mod_fare.predict(impute_subset[missing_fare])
    fare_imp.append(imputed_fare)

In [ ]:
# Replace missing values with mean of the imputed values
titanic_subset.loc[missing_age,'age'] = [np.mean(y) for y in zip(*age_imp)]
titanic_subset.loc[missing_fare,'fare'] = [np.mean(y) for y in zip(*fare_imp)]

In [ ]:
# Want to predict `survived`
y = titanic_subset.pop('survived')

Now that the data are complete, we scale the predictors and move on to model fitting.

In [ ]:
# Standardize the predictors
from sklearn import preprocessing
titanic_scaled = preprocessing.scale(titanic_subset)

### Support Vector Classifier
For both the SVC and the logistic regression model, we will use `GridSearchCV` to tune each model for optimal performance. For the SVC, we will tune the `C` parameter, which controls a penalty parameter (lower values indicate a higher degree of regularization), and the kernel function (radial basis function, third degree polynomial, or linear).

In [ ]:
# Fit SVM with cross validation
from sklearn import model_selection, svm
# Perform a grid search to tune the model and improve performance
from sklearn.model_selection import GridSearchCV

# Parameters to tune
svm_param_grid = {'C': [0.0001,0.001,0.01,0.1,0.5,1,2],
                  'kernel': ['rbf','poly','linear']}

In [ ]:
svm_mod = svm.SVC()
svm_cv = GridSearchCV(svm_mod, svm_param_grid, n_jobs=4, cv = 4).fit(titanic_scaled, y)
svm_cv.best_params_

Based on the above, the optimal parameters appear to be a radial basis kernel function and `C=0.1`. We will now fit an SVC with these parameters. We will use *accuracy* as the metric by which we assess fit. In class we used `f1-weighted`, which is a score method based on precision-recall. This approach works well when there is high class imbalance. Here, however, the imbalance is only around 60% to 40% (died to survived). We will use 4-fold cross validation to reduce overfitting.

In [ ]:
# Class imbalance
pd.crosstab(y,'survived')/len(y)

In [ ]:
# Fit the SVM based on the identified best parameters above
svc = svm.SVC(kernel='rbf', C=0.1)
svc_cv_scores = model_selection.cross_val_score(svc, titanic_scaled, y, cv=4,
                                            scoring='accuracy')
svc_cv_scores

In [ ]:
print("SVC CV Accuracy: %0.2f (+/- %0.2f)" % (svc_cv_scores.mean(), svc_cv_scores.std()*1.96))

### Logistic regression
For logistic regression, we will tune the penalty (L1 or L2) and the `C` parameter, which has a function similar to the SVC.

In [ ]:
# logistic regression
from sklearn.linear_model import LogisticRegression

lrm_param_grid = {'penalty': ['l1','l2'],
                 'C': [0.0001,0.001,0.01,0.1,0.5,1,2]}

In [ ]:
lrm_mod = LogisticRegression()

lrm_cv = GridSearchCV(lrm_mod, lrm_param_grid, n_jobs=4, cv=4).fit(titanic_scaled, y)
lrm_cv.best_params_

The best parameters were an L2 (Ridge) penalty with `C=0.001`. We fit a logistic regression model with these parameters and obtain the 4-fold cross validation accuracy estimates.

In [ ]:
# Now fit LRM based on best params
lrm = LogisticRegression(penalty='l2', C=0.001)

lrm_cv_scores = model_selection.cross_val_score(lrm, titanic_scaled, y, cv=4,
                                            scoring='accuracy')
lrm_cv_scores

In [ ]:
print("Logistic Regression CV Accuracy: %0.2f (+/- %0.2f)" % (lrm_cv_scores.mean(), lrm_cv_scores.std() * 2))

The average CV accuracy was 0.72, slightly lower than that of the SVC.

### Advantages/Disadvantages  

Logistic regression models (LRMs) have the advantage of being directly interpretable if assessing covariate effects is of interest, while an SVC is more of a "black box" method. In this way, LRMs can be preferable since they are simpler models, if you are not sacrificing too much in terms of model performance.

The SVM can use a variety of kernel functions to capture nonlinear effects in the data. A LRM, however, assumes a specific functional form. An SVC can also be more robust to outliers. One disadvantage of using a LRM is that multicollinearity of predictors should be taken into account. The models above each used the same predictors for comparison, but we did see that pclass and fare had a high negative association. Below is a plot that describes the relationship visually. SVC is fairly robust to multicollinearity, so predictions from the SVC may be more robust than the LRM.

In [ ]:
sns.stripplot(x="pclass", y="fare", data=titanic);
plt.show()

## Question 2

The file `TNNASHVI.txt` in your data directory contains daily temperature readings for Nashville, courtesy of the [Average Daily Temperature Archive](http://academic.udayton.edu/kissock/http/Weather/). This data, as one would expect, oscillates annually. Using PyMC3, use a Gaussian process to fit a non-parametric regression model to this data, choosing an appropriate covariance function. Plot 10 regression lines drawn from your process.

In [ ]:
daily_temps = pd.read_table("/Users/schluetd/Bios8366/data/TNNASHVI.txt", sep='\s+', 
                            names=['month','day','year','temp'], na_values=-99)
daily_temps.temp.plot(style='b.', figsize=(10,6), grid=False)

In [ ]:

#Drop the rows with na values
daily_temps_clean = daily_temps.dropna()

temps = daily_temps_clean.temp
x, y = temps.reset_index().values.T
X = x.reshape(-1, 1)

In [ ]:

with pm.Model() as sparse_model:
    
    l1 = pm.HalfCauchy("l1", 5)
    η1 = pm.HalfCauchy("η1", 5)
    l2 = pm.HalfCauchy("l2", 5)
    η2 = pm.HalfCauchy("η2", 5)
    
    # covariance 
    cov = (η1**2)*pm.gp.cov.Cosine(1, l1) + (η2**2) * pm.gp.cov.Matern52(1, l2)

    gp = pm.gp.MarginalSparse(cov_func=cov, approx="FITC")
    
    # set inducing points
    Xu = pm.gp.util.kmeans_inducing_points(50, X)
    
    # following 5.1 example
    σ = pm.HalfCauchy("σ", beta=2)
    obs = gp.marginal_likelihood("obs", X=X, Xu=Xu, y=y, sigma=σ)
    
    map_est = pm.find_MAP()

## For expediency, let's just compute the map
X_new = np.linspace(x.min(), x.max(), 600)[:,None]
with sparse_model:
    f_pred = gp.conditional("f_pred", X_new)
    pred_samples = pm.sample_ppc([map_est],
                                 vars=[f_pred], 
                                 samples=10)
    

fig = plt.figure(figsize=(13,7)); ax = fig.gca()
#plot the samples from the gp posterior with samples and shading
from pymc3.gp.util import plot_gp_dist
plot_gp_dist(ax, pred_samples["f_pred"], X_new)
#plot the data and the true latent function
plt.plot(X, y, 'ok', ms=3, alpha=0.5, label="Observed data")
plt.plot(Xu, 100*np.ones(Xu.shape[0]), "g.", ms=5, label="Inducing point location")
#axis labels and title
plt.xlabel("X")
plt.title("Posterior distribution over $f(x)$ at the observed values"); plt.legend();   



## Question 3

Fit a series of random-forest classifiers to the Wisconsin breast cancer dataset (`wisconsin_breast_cancer.csv`), to explore the sensitivity to the parameters `max_features`, the number of variables considered for splitting at each step, `max_depth`, the maximum depth of the tree, and `n_estimators`, the number of trees in the forest. Use appropriate metrics of performance, and include plots against a suitably-chosen range of values for these parameters.

Dataset description: Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image. Ten real-valued features are computed for each cell nucleus:

- `radius` (mean of distances from center to points on the perimeter) 
- `texture` (standard deviation of gray-scale values) 
- `perimeter` 
- `area` 
- `smoothness` (local variation in radius lengths) 
- `compactness` (perimeter^2 / area - 1.0) 
- `concavity` (severity of concave portions of the contour) 
- `concave points` (number of concave portions of the contour) 
- `symmetry` 
- `fractal dimension` ("coastline approximation" - 1)

The outcome to be predicted is tumor type (M = malignant, B = benign).

In [ ]:
wbc = pd.read_csv("/Users/schluetd/Bios8366/data/wisconsin_breast_cancer.csv")
wbc['malignant'] = wbc.diagnosis.replace({'M':1, 'B':0})
wisc_bc = wbc.drop('diagnosis', axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Pop off the outcome
X = wisc_bc.copy()
y = X.pop('malignant')

In [ ]:
# Class imbalance
pd.crosstab(y,'malignant')/len(y)

In [ ]:
from sklearn import model_selection

In [ ]:
# Function to fit a random forest given parameter values
def rf_sens(max_features, n_estimators, max_depth):
    rf_temp = RandomForestClassifier(max_features=max_features,
                                     n_estimators = n_estimators, 
                                     max_depth = max_depth,
                                     n_jobs=4)
    # Perform 4-fold cross validation
    cv_res = model_selection.cross_val_score(rf_temp, X, y, 
                                             cv=4, scoring='accuracy')
    
    # Return the mean cross-validation accuracy
    return cv_res.mean()

In [ ]:
# max_features sensitivity
max_feat_values = [1, 'sqrt', 10, 20, X.shape[1]]
mf_sens_highn_highd = [rf_sens(max_features=max_f, n_estimators=3000, max_depth=4) for max_f in max_feat_values]
mf_sens_lown_highd = [rf_sens(max_features=max_f, n_estimators=100, max_depth=4) for max_f in max_feat_values]
mf_sens_highn_lowd = [rf_sens(max_features=max_f, n_estimators=3000, max_depth=2) for max_f in max_feat_values]
mf_sens_lown_lowd = [rf_sens(max_features=max_f, n_estimators=100, max_depth=2) for max_f in max_feat_values]

In [ ]:
max_feat_values[1] = np.sqrt(X.shape[1])

plt.figure(figsize=(10,6))
l1=plt.plot(max_feat_values, mf_sens_highn_highd, label = 'n_estimators=3000,max_depth=4')
l2=plt.plot(max_feat_values, mf_sens_lown_highd, label = 'n_estimators=100,max_depth=4')
l3=plt.plot(max_feat_values, mf_sens_highn_lowd, label = 'n_estimators=3000,max_depth=2')
l4=plt.plot(max_feat_values, mf_sens_lown_lowd, label = 'n_estimators=100,max_depth=2')

plt.xlabel("max_features"); plt.ylabel("4-fold CV accuracy")
plt.title("max_features sensitivity"); 
plt.legend(['n_estimators=3000,max_depth=4',
                          'n_estimators=100,max_depth=4',
                          'n_estimators=3000,max_depth=2',
                          'n_estimators=100,max_depth=2'])

From the above we see that initially, as the maximum number of features considered grows, the accuracy improves. However, this pattern quickly plateaus and even turns to a decreasing pattern for some of the cases above. In particular, the cases that had a higher max_depth saw a more severe decline. These cases appeared to have higher accuracy for a lower value of max_features, while the cases with a lower max_depth peaked at max_features=20. 

However, note the y-axis. These trends are all within a two percentage point range. Generally, there doesn't appear to be extreme variation in performance as max_features varies. Empirical evidence has indicated that for classification problems, using max_features=sqrt(n_features) is a good option. It is interesting to note that this value was not optimal for all the situations presented above.

In [ ]:
# n_estimators sensitivity
n_est_values = [100,500,1000,3000,6000]
ne_sens_highmf_highd = [rf_sens(max_features=20, n_estimators=n_est, max_depth=4) for n_est in n_est_values]
ne_sens_lowmf_highd = [rf_sens(max_features='sqrt', n_estimators=n_est, max_depth=4) for n_est in n_est_values]
ne_sens_highmf_lowd = [rf_sens(max_features=20, n_estimators=n_est, max_depth=2) for n_est in n_est_values]
ne_sens_lowmf_lowd = [rf_sens(max_features='sqrt', n_estimators=n_est, max_depth=2) for n_est in n_est_values]

In [ ]:
plt.figure(figsize=(15,6))

l5=plt.plot(n_est_values, ne_sens_highmf_highd, label = 'max_features=20,max_depth=4')
l6=plt.plot(n_est_values, ne_sens_lowmf_highd, label = 'max_features=sqrt(n_features),max_depth=4')
l7=plt.plot(n_est_values, ne_sens_highmf_lowd, label = 'max_features=20,max_depth=2')
l8=plt.plot(n_est_values, ne_sens_lowmf_lowd, label = 'max_features=sqrt(n_features),max_depth=2')
plt.xlabel("n_estimators"); plt.ylabel("4-fold CV accuracy")
plt.title("n_estimators sensitivity"); 
plt.legend(['max_features=20,max_depth=4',
                          'max_features=sqrt(n_features),max_depth=4',
                          'max_features=20,max_depth=2',
                          'max_features=sqrt(n_features),max_depth=2']);
plt.show()

The plot shows a realtively constant trend of accuracy for each of the cases presented above as the number of estimators changes. The fits that had a larger max_depth tended to peak with a smaller number of estimators. While there was some minor fluctuation in accuracy as a function of n_estimators, the lines are all somewhat flat.

Again, note the relatively small difference in the maximum and minimum values of the y-axis.

#### `max_depth`  

In [ ]:
# max_depth sensitivity
max_depth_values = [1,2,3,4,5]
md_sens_highmf_highne = [rf_sens(max_features=20, n_estimators=3000, max_depth=max_dep) for max_dep in max_depth_values]
md_sens_lowmf_higne = [rf_sens(max_features='sqrt', n_estimators=3000, max_depth=max_dep) for max_dep in max_depth_values]
md_sens_highmf_lone = [rf_sens(max_features=20, n_estimators=100, max_depth=max_dep) for max_dep in max_depth_values]
md_sens_lowmf_lone = [rf_sens(max_features='sqrt', n_estimators=100, max_depth=max_dep) for max_dep in max_depth_values]

In [ ]:
plt.figure(figsize=(10,6))

l9=plt.plot(max_depth_values, md_sens_highmf_highne, label = 'max_features=20,n_estimators=3000')
l10=plt.plot(max_depth_values, md_sens_lowmf_higne, label = 'max_features=sqrt(n_features),n_estimators=3000')
l11=plt.plot(max_depth_values, md_sens_highmf_lone, label = 'max_features=20,n_estimators=100')
l12=plt.plot(max_depth_values, md_sens_lowmf_lone, label = 'max_features=sqrt(n_features),n_estimators=100')
plt.xlabel("max_depth"); plt.ylabel("4-fold CV accuracy")
plt.title("max_depth sensitivity"); 
plt.legend(['max_features=20,n_estimators=3000',
                          'max_features=sqrt(n_features),n_estimators=3000',
                          'max_features=20,n_estimators=100',
                          'max_features=sqrt(n_features),n_estimators=100']);
plt.show()

Overall, as the max_depth increases, the cross-validated accuracy appears to increase. The lines for all scenarios are largely in agreement, indicating that the number of estimators and max_features don't tend to alter the impact of max_depth. The accuracy increases by about 4% in all cases when comparing the lowest depth (1, i.e. no interactions) with the largest depth presented (5).

## Question 4

Use a grid search to optimize the number of estimators and max_depth for a Gradient Boosted Decision tree using the Wisconsin breast cancer data. Plug this optimal ``max_depth`` into a *single* decision tree.  Does this single tree over-fit or under-fit the data? Repeat this for the Random Forest.  Construct a single decision tree using the ``max_depth`` which is optimal for the Random Forest.  Does this single tree over-fit or under-fit the data?

In [46]:
wbc = pd.read_csv("/Users/schluetd/Bios8366/data/wisconsin_breast_cancer.csv")
wbc['malignant'] = wbc.diagnosis.replace({'M':1, 'B':0})
wisc_bc = wbc.drop('diagnosis', axis=1)
X = wisc_bc.copy()
y = X.pop('malignant')

# Parameter values to test
param_grid = {'n_estimators': [50,100,500,1000,3000,5000],
              'max_depth': [1,2,3,4,5]}

#### Gradient Boosted Tree

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Define the gradient boosted tree
gbt_est = GradientBoostingClassifier()
# Use 4-fold cross validation to pick the best parameters
gbt_cv = GridSearchCV(gbt_est, param_grid, n_jobs=4, cv=4).fit(X, y)

# Best hyperparameter settings
gbt_cv.best_params_

In [49]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
gbt = DecisionTreeClassifier(max_depth = 1)
gbt.fit(X,y)

In [50]:
from sklearn.metrics import confusion_matrix

In [ ]:
gbt_pred = gbt.predict(X)

pd.DataFrame(confusion_matrix(gbt_pred, y),
            index=['pred_benign','pred_malignant'],
            columns=['actual_benign','actual_malignant'])

This tree is likely overfit to the dataset because it is just a single instance of a decision tree. It does not incorporate any uncertainty or regularization, as we get from the gradient boosted tree. The training accuracy is relatively high, though it is not a perfect classifier. Perfect accuracy on a training dataset is an almost sure indication of overfitting.

#### Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Fit the random forest classifier
rfc_est = RandomForestClassifier()
rfc_cv = GridSearchCV(rfc_est, param_grid, n_jobs=4, cv=4).fit(X, y)

# best hyperparameter setting
rfc_cv.best_params_

In [ ]:
rfc = DecisionTreeClassifier(max_depth = 5)
rfc.fit(X,y)

In [ ]:
rfc_pred = rfc.predict(X)

pd.DataFrame(confusion_matrix(rfc_pred, y),
            index=['pred_benign','pred_malignant'],
            columns=['actual_benign','actual_malignant'])

This tree is definitely overfit to the data. Its training accuracy is far too high, and this same tree would probably not do well generalizing to a new set of observations. Random forests are like averaging over many decision trees. By fitting one alone, we lose any way of accounting for that uncertainty. A single tree is rarely ever a good predictor as it may be difficult to have good accuracy on a validation dataset.